# Beispiel 3.8: Nicht-Schlüsselkomponenten aus Elementbilanz
Bearbeitet von Franz Braun

Dieses Beispiel befindet sich im Lehrbuch auf den Seiten 30 - 31. Die Nummerierung
der verwendeten Gleichungen entspricht der Nummerierung im Lehrbuch. Das hier angewendete
Vorgehen entspricht dem im Lehrbuch vorgestellten Lösungsweg.

Zunächst werden die benötigten Pakete importiert.

In [9]:
### Import
import numpy as np
from scipy.optimize import root
from tabulate import tabulate

Für das Beispiel der Synthesegaserzeugung aus Methan und Wasserdampf (vgl. Beispiele 3.6 und 3.7) lässt sich die Element-Spezies-Matrix $B$ nach Kapitel 3.6 (Seite 28 - 31) formulieren. 

\begin{align*}
&\mathrm{1.\,\, Reaktion} \quad  &                   \mathrm{CH_4 + H_2O}    &      \leftrightarrow     \mathrm{CO + 3H_2} \\
&\mathrm{2.\,\, Reaktion} \quad  &                   \mathrm{CO + H_2O}      &      \leftrightarrow     \mathrm{CO_2 + H_2} \\
&\mathrm{3.\,\, Reaktion} \quad  &                   \mathrm{CH_4}           &      \leftrightarrow     \mathrm{C + 2\,H_2} \\
&\mathrm{4.\,\, Reaktion} \quad  &                   \mathrm{2\, CH_4}       &      \leftrightarrow     \mathrm{C_2H_6 + H_2} \\
&\mathrm{5.\,\, Reaktion} \quad  &                   \mathrm{C + H_2O}       &      \leftrightarrow     \mathrm{CO + H_2} \\
&\mathrm{6.\,\, Reaktion} \quad  &                   \mathrm{2\,CO}          &      \leftrightarrow     \mathrm{C + CO_2} \\
\end{align*}

Diese Matrix wird in die Teilmatrizen $B_1$ der Schlüsselkomponenten und $B_2$ für Nicht-Schlüsselkomponenten unterteilt. 

In [10]:
                        #   H2  CO  H2O CH4 C   CO2 C2H6
B           = np.array([[   0,  1,  0,  1,  1,  1,  2   ],  # C
                        [   2,  0,  2,  4,  0,  0,  6   ],  # H
                        [   0,  1,  1,  0,  0,  2,  0   ]]) # O

# Schlüsselkomponenten                       
                       #   H2  CO  H2O CH4 
B_1         = np.array([[   0,  1,  0,  1   ],  # C
                        [   2,  0,  2,  4   ],  # H
                        [   0,  1,  1,  0   ]]) # O

# Nicht- Schlüsselkomponenten
                        #    C   CO2 C2H6
B_2         = np.array([[   1,  1,  2   ],  # C
                        [   0,  0,  6   ],  # H
                        [   0,  2,  0   ]]) # O 

Mit der Gleichung 3.30 werden die Umsätze der Nicht-Schlüsselkomponenten bestimmt. Hierfür wird analog zum Beispiel 3.7 ein Vektorprodukt $MP$ definiert:

\begin{align*}
MP = - B_{2}^{-1} \, B_{1}.
\end{align*}

Die inverse Matrix $B_{2}^{-1}$ wird zuvor in einem separaten Schritt aus $B_{2}$ gebildet.

Aus Gründen der Anschaulichkeit werden vor der tabellarischen Ausgabe von $MP$ zur Bildung der Komponentenschreibweise alle $"-0"$ Einträge in $MP$ durch "0" mit zwei For-Schleifen ersetzt. Jede Tabellenzeile entspricht der Stoffmengenänderung (bzw. Umsatz) einer Nicht-Schlüsselkomponente. In den Spalten stehen die Umsätze der Schlüsselkomponenten.

In [11]:
B_2_inv     = np.linalg.inv(B_2)            # Bildung der inversen Matrix B_2^-1
print('B_2^-1 :', B_2_inv)

MP          = - np.matmul(B_2_inv, B_1)     # Vektorprodukt
print('MP :', MP)
print()
print()

# Austauschen von "-0" mit "0" in der Matrix 
for i in range(len(MP)):
    for j in range(len(MP[0])):
        if MP[i][j] == -0:
            MP[i][j] = 0



# Ausgabe der Ergebnisse als Tabelle

names_y     = np.array([['Delta_n_C / mol'], ['Delta_n_CO2 / mol'], ['Delta_n_C2H6 / mol']])


# Array für Tabelle
table       = np.hstack((names_y, np.round(MP, 4)))

print(tabulate(table,headers = ['Delta_n_H2 / mol', 'Delta_n_CO / mol', 'Delta_n_H2O / mol', 'Delta_n_CH4 / mol']))

B_2^-1 : [[ 1.         -0.33333333 -0.5       ]
 [ 0.          0.          0.5       ]
 [ 0.          0.16666667  0.        ]]
MP : [[ 0.66666667 -0.5         1.16666667  0.33333333]
 [-0.         -0.5        -0.5        -0.        ]
 [-0.33333333 -0.         -0.33333333 -0.66666667]]


                      Delta_n_H2 / mol    Delta_n_CO / mol    Delta_n_H2O / mol    Delta_n_CH4 / mol
------------------  ------------------  ------------------  -------------------  -------------------
Delta_n_C / mol                 0.6667                -0.5               1.1667               0.3333
Delta_n_CO2 / mol               0                     -0.5              -0.5                  0
Delta_n_C2H6 / mol             -0.3333                 0                -0.3333              -0.6667


Aus der obigen Tabelle ergibt sich folgende Komponentenschreibweise für die Umsätze der Nicht-Schlüsselkomponenten:

\begin{align*}
\Delta n_\mathrm{C} &= \frac{2}{3}\, \Delta n_\mathrm{H_2} - \frac{1}{2}\, \Delta n_\mathrm{CO} + \frac{7}{6}\, \Delta n_\mathrm{H_2O} + \frac{1}{3}\, \Delta n_\mathrm{CH_4},\\
\Delta n_\mathrm{CO_2} &= - \frac{1}{2}\, \Delta n_\mathrm{CO} - \frac{1}{2}\, \Delta n_\mathrm{H_2O},\\
\Delta n_\mathrm{C_2H_6} &= - \frac{1}{3}\, \Delta n_\mathrm{H_2} - \frac{1}{3}\, \Delta n_\mathrm{H_2O} - \frac{2}{3}\, \Delta n_\mathrm{CH_4}.
\end{align*}

Zahlenbeispiel: Wir nehmen an, dass folgende Umsätze für die Schlüsselkomponenten $\mathrm{H_2}$, $\mathrm{CO}$, $\mathrm{H_2O}$ und $\mathrm{CH_4}$ gemessen wurden: 

\begin{align*}
\Delta n_1 = \left[\begin{array}{c}30\\ 9\\ -10\\ -10\end{array}\right]\,\mathrm{mol}
\end{align*}

In [12]:
Delta_n_1 = np.array([[ 30,  # H2
                         9,  # CO
                       -10,  # H2O
                       -10]]) # CH4

Delta_n_2 = np.matmul(MP, Delta_n_1.T) # Vektorprodukt

print('Delta_n_2 :', Delta_n_2)

Delta_n_2 : [[ 5.0000000e-01]
 [ 5.0000000e-01]
 [-8.8817842e-16]]


Es ergeben sich für die Nicht-Schlüsselkomponenten nun folgende Umsätze:

\begin{align*}
\Delta n_2 = \left[\begin{array}{c}0,5\\ 0,5\\ 0\end{array}\right]\,\mathrm{mol}
\end{align*}

Diese Ergebnisse sind erwartungsgemäß identisch mit Beispiel 3.7, in dem die gleichen Umsätze der Schlüsselkomponenten genutzt werden.